# Assignment 4: CNN and ResNet


We first import all we need

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Subset

import matplotlib.pyplot as plt
import numpy as np

import torchvision
import torchvision.datasets as DS
import torchvision.transforms as transforms
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

torch.cuda.is_available()
cuda = torch.cuda.is_available()

In [2]:
file_path = 'uw-cs480-winter23/'
image_path = 'uw-cs480-winter23/noisy-images/noisy-images/'

def load_data():
    train_df = pd.read_csv(file_path + 'train.csv')
    test_df = pd.read_csv(file_path + 'test.csv')
    return train_df, test_df

In [3]:
train_df, test_df = load_data()
data_size = len(train_df)
print(data_size)

# remove all free gifts
train_df = train_df[train_df.category != 'Free Gifts']
data_size = len(train_df)
print(data_size)

categories = train_df.category.unique()
category_d = {k: v for v, k in enumerate(categories)}

genders = train_df.gender.unique()
gender_d = {k: v for v, k in enumerate(genders)}
num_genders = len(genders)

baseColours = train_df.baseColour.unique()
baseColour_d = {k: v for v, k in enumerate(baseColours)}
num_baseColours = len(baseColours)

seasons = train_df.season.unique()
season_d = {k: v for v, k in enumerate(seasons)}
num_seasons = len(seasons)

usages = train_df.usage.unique()
usage_d = {k: v for v, k in enumerate(usages)}
num_usages = len(usages)

# training data

train_df.replace(
    {'category': category_d}
    , inplace=True
)

# testing data

test_df.replace(
    {'category': category_d}
    , inplace=True
)

id_target = train_df[['id', 'category']].values

21627
21583


In [4]:
train_labels = id_target[:, 1]
train_id = id_target[:, 0]

In [5]:
train_labels_onehot = []

for (idx, cate) in enumerate(train_labels):
    train_labels_onehot.append(np.eye(26)[cate])

In [6]:
# id_target[:, 1] = train_labels_onehot

new_id_target = np.array([train_id, train_labels_onehot]).T

C:\Users\zhc17\AppData\Local\Temp\ipykernel_49016\4178741031.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  new_id_target = np.array([train_id, train_labels_onehot]).T


In [7]:
transformer = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

class Image_Dataset(Dataset):

    def __init__(self, id_target, folder=image_path):
        self.id_target = id_target
        self.folder = folder
        self.transform = transformer
    def __len__(self):
        return len(self.id_target)

    def __getitem__(self, idx):
        img_name = self.folder + str(self.id_target[idx][0]) + '.jpg'
        image = Image.open(img_name)
        result = self.transform(image)

        return result, self.id_target[idx][1]

In [8]:
train_image = Image_Dataset(new_id_target)
test_image = Image_Dataset(new_id_target)

In [9]:

train_set, test_set = train_test_split(train_image, test_size=0.2, shuffle=True)

In [10]:
# create data loaders
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

batch_size = 100
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [11]:
for images, labels in train_loader:
    print(images.shape)
    break

torch.Size([100, 3, 80, 60])


In [12]:
class ANNClassifier(nn.Module):
    def __init__(self):
        super(ANNClassifier, self).__init__()
        self.fc1 = nn.Linear(3*60*80, 128)
        self.fc2 = nn.Linear(128, 26)

    def forward(self, x):
        x = x.view(-1, 3*60*80)  # Flatten the input image
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### Task 2: Implementing the Test Loop

In [13]:
def test(model, loss_function, device):
    # we first move our model to the configured device
    model = model.to(device = device)

    # we make sure we are not tracking gradient
    # gradient is used in training, we do not need it for test
    with torch.no_grad():
        risk = 0
        accuracy = 0

        # loop over test mini-batches
        for i, (images, labels) in enumerate(test_loader):
            # reshape labels to have the same form as output
            # make sure labels are of torch.float32 type
            labels = labels.float()

            # move tensors to the configured device
            images = images.to(device = device)
            labels = labels.to(device = device)

            # forward pass
            outputs = model(images)
            loss = loss_function(outputs, labels)

            
            # compute the fraction of correctly predicted labels
            # correct_predict = torch.sum(labels- outputs).item()
            predicted_classes = torch.argmax(labels, dim=1)
            selected_probs = outputs[torch.arange(len(outputs)), predicted_classes]

            risk += loss.item()
            accuracy += selected_probs.mean().item()

        # average test risk and accuracy over the whole test dataset
        test_risk = risk/i
        test_accuracy = accuracy/i

    return test_risk, test_accuracy

Now test your untrained model.

### Task 3: Implementing Training Loop

In [14]:
def train(model, num_epochs, device):
    # we first move our model to the configured device
    model = model.to(device = device)

    # set loss to binary CE
    loss_function = nn.BCELoss()

    # Set optimizer with optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Initiate the values
    train_risk = []
    test_risk = []
    test_accuracy = []

    for epoch in range(num_epochs):
        # training risk in one epoch
        risk = 0

        # loop over training data
        for i, (images, labels) in enumerate(train_loader):

            # reshape labels to have the same form as output
            # make sure labels are of torch.float32 type
            labels = labels.float()

            # move tensors to the configured device
            images = images.to(device = device)
            labels = labels.to(device = device)

            # forward pass
            outputs = model(images)
            loss = loss_function(outputs, labels)

            # collect the training loss
            risk += loss.item()

            # backward pass
            optimizer.zero_grad()
            loss.backward()
            # use auto-grad (just 1 line)

            # one step of gradient descent
            optimizer.step()

        # test out model after update by the optimizer
        risk_epoch, accuracy_epoch = test(model, loss_function, device)

        # collect losses and accuracy
        train_risk.append(risk/i)
        test_risk.append(risk_epoch)
        test_accuracy.append(accuracy_epoch)

        # we can print a message every second epoch
        if (epoch+1) % 2 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_risk[-1]}, Test Loss: {test_risk[-1]}, Test Accuracy: {test_accuracy[-1]}")

    # plot the losses
    plt.plot([i+1 for i in range(num_epochs)], train_risk, label="train")
    plt.plot([i+1 for i in range(num_epochs)], test_risk, label="test")
    plt.legend()
    plt.show()

    # plot the accuracy
    plt.plot([i+1 for i in range(num_epochs)], test_accuracy)
    plt.show()

    return train_risk, train_risk, test_accuracy

Now, we can train

In [15]:
model = ANNClassifier()

if torch.has_cuda:
    device = "cuda"
elif torch.has_mps:
    device = "mps"
else:
    device = "cpu"

train_risk, train_risk, test_accuracy = train(model, 20, device)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
